In [1]:
(ql:quickload :kenzo)

To load "kenzo":
  Load 1 ASDF system:
    kenzo
; Loading "kenzo"



(:KENZO)

In [2]:
(cat:cat-init)


---done---

NIL

### 2.2.1 Representation of a reduction

In [3]:
(documentation 'cat:reduction 'type)

"----------------------------------------------------------------[class-doc]
REDUCTION
Slots: (tcc bcc f g h idnm orgn)
Instances of this class represent reductions of chain complexes.

     ^    h     s ^                 f ° g = 1
     C  ----->    C                          C
      ^                     h ° d + d ° h = 1  - g ° f
    | |                                      ^
  f | | g                                    C
    | |                             f ° h = 0
    v                               h ° g = 0
     C                              h ° h = 0

The class has 7 slots:
                                                               ^
1. TCC, an object of type CHAIN-COMPLEX, the top chain complex C. Its reader
   function is TCC1.

2. BCC, an object of type CHAIN-COMPLEX, the bottom chain complex C. Its reader
   function is BCC1.

3. F, an object of type MORPHISM representing the morphism f. Its reader
   function is F1.

4. G, an object of type MORPHISM representing the m

### 2.2.2 The function $\texttt{build-rdct}$

In [4]:
(documentation 'cat:build-rdct 'function)

"--------------------------------------------------------------[function-doc]
BUILD-RDCT
Args: (&key f g h orgn)
Returns an instance of the class REDUCTION. The keyword arguments are as
follows:

:F F, an object of type morphism representing the morphism f of a reduction

:G G, an object of type morphism representing the morphism g of a reduction

:H H, an object of type morphism representing the morphism h of a reduction

:ORGN ORGN, a list containing a relevant and carefully chosen comment about
            the origin of the chain complex. This comment should be unique
            for a Kenzo session (between calls of CAT-INIT), as it is used
            for caching purposes.

Use this function instead of creating instances via the standard constructor
MAKE-INSTANCE.
------------------------------------------------------------------------------"

### 2.2.4 Verification functions

In [5]:
(documentation 'cat:pre-check-rdct 'function)

"--------------------------------------------------------------[function-doc]
PRE-CHECK-RDCT
Args: (rdct)
Assigns the following Lisp global variables, using morphisms from RDCT and the
differentials and identity morphisms of the underlying chain complexes,
according to these formulas:

*TDD*         = d  ° d
                 ^    ^
                 C    C

*BDD*         = d  ° d
                 C    C

*ID-FG*       = Id  - f ° g
                  C

*ID-GF-DH-HD* = Id  - g ° f - (d ° h + h ° d )
                  ^             ^           ^
                  C             C           C

*HH*          = h ° h
*FH*          = f ° h
*HG*          = h ° g

*DF-FD*       = d  ° f - f ° d
                 C            ^
                              C

*DG-GD*       = d  ° g - g ° d
                 ^            C
                 C

------------------------------------------------------------------------------"

In [6]:
(documentation 'cat:check-rdct-no-wait 'function)

"--------------------------------------------------------------[function-doc]
CHECK-RDCT-NO-WAIT
Args: ()
Maps combinations of the top and bottom chain complexes of a reduction using
the morphisms created by PRE-CHECK-RDCT. Having no parameters, the
combinations must be provided via the Lisp global variables *TC* and *BC*,
for the top and bottom chain complex, respectively. If the morphisms are
coherent, the result of each mapping is a null combination.
------------------------------------------------------------------------------"

#### Example

In [7]:
(defun cdelta (dmns)
    (cat:build-chcm
     :cmpr #'cat:l-cmpr
     :basis :locally-effective
     :bsgn '(0)
     :intr-dffr
         #'(lambda (degr gmsm)
            (cat:make-cmbn
             :degr (1- degr)
             :list (do ((rslt cat:+empty-list+
                              (cons (cons sign
                                          (append
                                           (subseq gmsm 0 nark)
                                           (subseq gmsm (1+ nark))))
                                    rslt))
                        (sign 1 (- sign))
                        (nark 0 (1+ nark)))
                       ((> nark degr) rslt))))
     :strt :gnrt
     :orgn `(locally effective version of C_* delta ,dmns)))

CDELTA

In [8]:
(defun make-f (tdmns bdmns)
    (cat:build-mrph
     :sorc (cdelta tdmns) :trgt (cdelta bdmns) :degr 0
     :intr #'(lambda (degr gmsm)
                (let ((pos (position-if #'(lambda (vertex) (>= vertex bdmns))
                                        gmsm)))
                     (if pos
                         (if (< pos degr)
                             (cat:zero-cmbn degr)
                             (cat:cmbn degr 1 (nconc (butlast gmsm) (list bdmns))))
                         (cat:cmbn degr 1 gmsm))))
     :strt :gnrt
     :orgn `(projection delta ,tdmns => delta ,bdmns)))

MAKE-F

In [9]:
(defun make-g (tdmns bdmns)
    (cat:build-mrph
     :sorc (cdelta bdmns) :trgt (cdelta tdmns) :degr 0
     :intr #'identity
     :strt :cmbn
     :orgn `(injection delta ,bdmns => delta ,tdmns)))

MAKE-G

In [10]:
(defun make-h (tdmns bdmns)
    (cat:build-mrph
     :sorc (cdelta tdmns) :trgt (cdelta tdmns) :degr +1
     :intr #'(lambda (degr gmsm)
                (let ((pos (position-if #'(lambda (vertex) (>= vertex bdmns))
                                        gmsm)))
                     (if pos
                         (if (member bdmns gmsm)
                             (cat:zero-cmbn (1+ degr))
                             (cat:cmbn (1+ degr) (cat:-1-expt-n pos)
                                       (append (subseq gmsm 0 pos) (list bdmns)
                                               (subseq gmsm pos))))
                         (cat:zero-cmbn (1+ degr)))))
     :strt :gnrt
     :orgn `(homotopy for delta ,tdmns => ,bdmns)))

MAKE-H

In [11]:
(defun make-rdct (tdmns bdmns)
    (let ((rdct (cat:build-rdct
                 :f (make-f tdmns bdmns)
                 :g (make-g tdmns bdmns)
                 :h (make-h tdmns bdmns)
                 :orgn `(reduction delta ,tdmns ,bdmns))))
         rdct))

MAKE-RDCT

In [12]:
(cat:def rdct (make-rdct 6 3))

[K8 Reduction K1 => K3]

In [13]:
(cat:orgn rdct)

(REDUCTION DELTA 6 3)

In [14]:
(cat:pre-check-rdct rdct)


---done---

NIL

In [15]:
(setf cat:*tc* (cat:cmbn 2 1 '(0 1 2) 10 '(1 2 3) 100 '(1 2 4) 1000 '(2 3 4)))


----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------


In [16]:
(setf cat:*bc* (cat:cmbn 3 4 '(0 1 2 3)))


----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------


In [17]:
(cat:check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
R

NIL

In [18]:
(cat:def trdct (make-rdct 6 4))

[K35 Reduction K1 => K30]

In [19]:
(cat:def brdct (make-rdct 4 3))

[K39 Reduction K30 => K3]

In [20]:
(setf rdct (cat:cmps brdct trdct))

[K45 Reduction K1 => K3]

In [21]:
(cat:pre-check-rdct rdct)


---done---

NIL

In [22]:
(cat:check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
R

NIL